# INTEGRADOR  DE LOS EVENTOS DE PARTIDOS

In [ ]:
### lo que se llevará a cabo con el siguiente codigo es:
    1º lectura de los 400 archivos csv de los eventos de cada partido
    2º Concatenarlos (se complica ya que no todos los archivos tienen la misma estructura de columnas, depende de los eventos de cada partido)
    3º Se aplica el metricas_funciones.py en este notebook con el fin de ya tener un df_final con todo lo necesario para actualizar la base de datos.
    4º Se añade a la base de datos en la tabla FAC_EVENT para eso se descarga el archivo csv ya que el integrador de python daba error fast_execution
    5º Se añade a Azure utilizando la herramienta de exportación e importación SQL

In [1]:
import pandas as pd
import glob
import os
import pandas as pd
from socceraction.data.statsbomb import StatsBombLoader
import socceraction.spadl as spadl
import socceraction.xthreat as xthreat
import matplotsoccer as mps
import numpy as np
import matplotlib.pyplot as plt
from socceraction.xthreat import load_model, get_successful_move_actions
import json
from mplsoccer import VerticalPitch
import statsmodels.formula.api as smf
import statsmodels.api as sm
import pickle 

ruta_archivos = r'G:\Mi unidad\TFM\OPTA\0-Python\scoresway\output'
patron = os.path.join(ruta_archivos, '*_eventData.csv')
archivos_eventData = glob.glob(patron)

print(f"Total de archivos encontrados: {len(archivos_eventData)}")

dataframes = []
errores = []

for archivo in archivos_eventData:
    try:
        # Leer usando ; como separador
        df = pd.read_csv(archivo, sep=';', encoding='utf-8', on_bad_lines='skip')
        df['source_file'] = os.path.basename(archivo)
        dataframes.append(df)
    except Exception as e:
        print(f"❌ Error al leer {archivo}")
        print(f"   Motivo: {e}")
        errores.append(archivo)

# Concatenar aunque tengan columnas diferentes
df_concatenado = pd.concat(dataframes, ignore_index=True, sort=False)

print(f"\n✅ Archivos cargados correctamente: {len(dataframes)}")
print(f"⚠️ Archivos con error: {len(errores)}")
print("\n🔍 Primeras filas del DataFrame:")


Total de archivos encontrados: 398

✅ Archivos cargados correctamente: 398
⚠️ Archivos con error: 0

🔍 Primeras filas del DataFrame:


In [2]:
# Lista de columnas a mantener (solo las claves del diccionario)
columnas_a_mantener = [
    'assist', 'keyPass','value_LongBall', 'outcome', 'periodId', 'timeMin',
    'value_GoalShotTimestamp', 'value_LeftFooted',
    'value_Minutes', 'value_OppositeRelatedEventId', 'value_RightFooted',
    'endX', 'endY', 'eventId', 'id', 'isFirstEleven', 'isGoal',
    'matchId', 'minute', 'oppositionTeamName', 'playerId', 'playerName',
    'position', 'second', 'teamId', 'teamName', 'time_seconds',
    'type_displayName', 'value_Angle', 'value_Assisted', 'value_Blocked',
    'value_CornerTaken', 'value_Cross', 'value_Defensive', 'value_Foul',
    'value_GoalKick', 'value_Hands', 'value_Head', 'value_HeadPass',
    'value_IndividualPlay', 'value_IntentionalAssist', 'value_KeeperThrow',
    'value_LayOff', 'value_Length', 'value_Offensive', 'value_PassEndX',
    'value_PassEndY', 'value_RegularPlay', 'value_RelatedEventId',
    'value_Zone', 'x', 'y'
]

# Creamos el nuevo DataFrame filtrado
df_filtrado = df_concatenado[columnas_a_mantener].copy()
df_filtrado

,assist,keyPass,value_LongBall,outcome,periodId,timeMin,value_GoalShotTimestamp,value_LeftFooted,value_Minutes,value_OppositeRelatedEventId,...,value_LayOff,value_Length,value_Offensive,value_PassEndX,value_PassEndY,value_RegularPlay,value_RelatedEventId,value_Zone,x,y
0,NaN,NaN,NaN,1,16,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0,0","0,0"
1,NaN,NaN,NaN,1,16,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0,0","0,0"
2,NaN,NaN,NaN,1,1,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0,0","0,0"
3,NaN,NaN,NaN,1,1,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0,0","0,0"
4,NaN,NaN,NaN,1,1,0,NaN,NaN,NaN,NaN,...,NaN,"18,8",NaN,"32,2","53,6",NaN,NaN,NaN,"49,9","49,9"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
679201,NaN,NaN,NaN,1,2,96,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0,0","0,0"
679202,NaN,NaN,NaN,1,14,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0,0","0,0"
679203,NaN,NaN,NaN,1,14,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0,0","0,0"
679204,NaN,NaN,NaN,1,14,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0,0","0,0"


In [3]:
diccionario_columnas_final = {
    # Renombradas (una sola vez cada destino)
    'assist': 'value_ShotAssist',
    'keyPass': 'value_KeyPass',
    'outcome': 'outcomeType_value',
    'periodId': 'period_value',
    'second': 'second',  
    'value_IntentionalAssist': 'value_IntentionalAssist',  
    'value_Blocked': 'value_Blocked', 
    'value_GoalShotTimestamp': 'value_GoalShotTimestamp',
    'value_LeftFooted': 'value_LeftFoot',
    'value_LongBall': 'value_Longball',  
    'value_OppositeRelatedEventId': 'value_OppositeRelatedEvent',
    'value_RightFooted': 'value_RightFoot',

    # Ya correctas (se renombran a sí mismas)
    'endX': 'endX',
    'endY': 'endY',
    'eventId': 'eventId',
    'id': 'id',
    'isFirstEleven': 'isFirstEleven',
    'isGoal': 'isGoal',
    'matchId': 'matchId',
    'minute': 'minute',
    'oppositionTeamName': 'oppositionTeamName',
    'playerId': 'playerId',
    'playerName': 'playerName',
    'position': 'position',
    'teamId': 'teamId',
    'teamName': 'teamName',
    'time_seconds': 'time_seconds',
    'type_displayName': 'type_displayName',
    'value_Angle': 'value_Angle',
    'value_Assisted': 'value_Assisted',
    'value_CornerTaken': 'value_CornerTaken',
    'value_Cross': 'value_Cross',
    'value_Defensive': 'value_Defensive',
    'value_Foul': 'value_Foul',
    'value_GoalKick': 'value_GoalKick',
    'value_Hands': 'value_Hands',
    'value_Head': 'value_Head',
    'value_HeadPass': 'value_HeadPass',
    'value_IndividualPlay': 'value_IndividualPlay',
    'value_KeeperThrow': 'value_KeeperThrow',
    'value_LayOff': 'value_LayOff',
    'value_Length': 'value_Length',
    'value_Offensive': 'value_Offensive',
    'value_PassEndX': 'value_PassEndX',
    'value_PassEndY': 'value_PassEndY',
    'value_RegularPlay': 'value_RegularPlay',
    'value_RelatedEventId': 'value_RelatedEventId',
    'value_Zone': 'value_Zone',
    'x': 'x',
    'y': 'y'
}

# Renombrar columnas en df_filtrado
df_renombrado = df_filtrado.rename(columns=diccionario_columnas_final)
df_renombrado.head(50)

,value_ShotAssist,value_KeyPass,value_Longball,outcomeType_value,period_value,timeMin,value_GoalShotTimestamp,value_LeftFoot,value_Minutes,value_OppositeRelatedEvent,...,value_LayOff,value_Length,value_Offensive,value_PassEndX,value_PassEndY,value_RegularPlay,value_RelatedEventId,value_Zone,x,y
0,NaN,NaN,NaN,1,16,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0,0","0,0"
1,NaN,NaN,NaN,1,16,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0,0","0,0"
2,NaN,NaN,NaN,1,1,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0,0","0,0"
3,NaN,NaN,NaN,1,1,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0,0","0,0"
4,NaN,NaN,NaN,1,1,0,NaN,NaN,NaN,NaN,...,NaN,"18,8",NaN,"32,2","53,6",NaN,NaN,NaN,"49,9","49,9"
5,NaN,NaN,NaN,1,1,0,NaN,NaN,NaN,NaN,...,NaN,"17,9",NaN,"33,6","79,6",NaN,NaN,NaN,"32,3","53,3"
6,NaN,NaN,NaN,1,1,0,NaN,NaN,NaN,NaN,...,NaN,"11,0",NaN,"34,1","95,7",NaN,NaN,NaN,"33,6","79,6"
7,NaN,NaN,NaN,1,1,0,NaN,NaN,NaN,NaN,...,NaN,"15,6",NaN,"22,5","81,4",NaN,NaN,NaN,"34,1","95,7"
8,NaN,NaN,"1,0",0,1,0,NaN,NaN,NaN,NaN,...,NaN,"50,5",NaN,"70,5","86,5",NaN,NaN,NaN,"22,5","81,4"
9,NaN,NaN,"1,0",0,1,0,NaN,NaN,NaN,NaN,...,NaN,"17,1",NaN,"53,6","11,3",NaN,NaN,NaN,"37,3","12,8"


limit_output extension: Maximum message size of 10000 exceeded with 22686 characters

In [4]:
def preparar_datos(df):
    # Rellenar NaN antes de convertir a string para evitar 'nan' en string
    df['endX'] = df['endX'].fillna(0)
    df['endY'] = df['endY'].fillna(0)

    # Ahora convierto a string y reemplazo ',' por '.' para convertir a float
    df['x'] = df['x'].astype(str).str.replace(',', '.').astype(float)
    df['y'] = df['y'].astype(str).str.replace(',', '.').astype(float)
    df['endX'] = df['endX'].astype(str).str.replace(',', '.').astype(float)
    df['endY'] = df['endY'].astype(str).str.replace(',', '.').astype(float)

    df['time_seconds'] = pd.to_numeric(df['time_seconds'], errors='coerce')

    df['minute'] = df['minute'].astype(str).str.replace(',', '.', regex=False).astype(float)
    df['second'] = df['second'].astype(str).str.replace(',', '.', regex=False).astype(float)

    df['Segundos'] = (df['minute'] * 60 + df['second']).fillna(0).astype(int)

    df['x_real'] = df['x'] * 1.05      # 100 -> 105 m
    df['y_real'] = df['y'] * 0.68      # 100 -> 68 m
    df['endx_real'] = df['endX'] * 1.05
    df['endy_real'] = df['endY'] * 0.68

    df['ZonaX'] = (df['x_real'] // 13.125).astype(int)
    df['ZonaY'] = (df['y_real'] // 13.6).astype(int)

    df['Zona_end_X'] = (df['endx_real'] // 13.125).astype(int)
    df['Zona_end_Y'] = (df['endy_real'] // 13.6).astype(int)
    df['outcomeType_displayName'] = df['outcomeType_value'].map({1: 'Successful', 0: 'Unsuccessful'})

    return df

# Métricas de pase
def calcular_pases(df):
    df['LongPass'] = ((df['type_displayName'] == 'Pass') & (df['value_Longball'] == '1,0')).astype(int)
    df['ShortPass'] = ((df['type_displayName'] == 'Pass') & (df['value_Longball'].isna())).astype(int)
    
    # Key passes
    id_evento_keypasses = df.loc[df['value_Assisted'] == '1,0', 'value_RelatedEventId']
    df['KeyPasses'] = df['eventId'].isin(id_evento_keypasses).astype(int)

    # Progressive passes
    df['ProgressPasses'] = (
        (df['type_displayName'] == 'Pass') &
        (df['outcomeType_value'] == 1) &
        (df['x'] < df['endX']) &
        (
            ((df.endX - df.x) * 105 / 100 > 30) & (df.endX < 50) |
            ((df.endX - df.x) * 105 / 100 > 15) & (df.endX > 50) & (df.x <= 50) |
            ((df.endX - df.x) * 105 / 100 > 10) & (df.endX > 50) & (df.x >= 50)
        )
    ).astype(int)

    return df

# Toques en área rival y propia
def calcular_toques_area(df):
    tipos_validos = [
        'Aerial',
        'Attempt Saved',
        'Ball touch',
        'Blocked Pass',
        'Claim',
        'Clearance',
        'Dispossessed',
        'Goal',
        'Interception',
        'Keeper pick-up',
        'Miss',
        'Offside Pass',
        'Save',
        'Tackle',
        'Take On',
        'Smother',
        'Good skill',
        'Punch'
    ]

    df['TouchesRivalArea'] = (
        (~((df['type_displayName'] == 'Pass') & (df.get('value_KeeperThrow', 0) == 1))) &
        (df['type_displayName'].isin(tipos_validos)) &
        (df['x'] > 83) & (df['endX'] < 100) &
        (df['y'] > 21.1) & (df['endY'] < 78.9)
    ).astype(int)

    df['TouchesOwnArea'] = (
        (~((df['type_displayName'] == 'Pass') & (df.get('value_KeeperThrow', 0) == 1))) &
        (df['type_displayName'].isin(tipos_validos)) &
        (df['x'] < 17) & (df['endX'] > 0) &
        (df['y'] > 21.1) & (df['endY'] < 78.9)
    ).astype(int)

    return df

# Modelo xThreat
def aplicar_xT(df):
    df['isShot'] = df['type_displayName'].isin(['Goal', 'Attempt Saved', 'Miss', 'Post']).astype(int)
    df_xT = df[(df['isShot'] == True) | (df['type_displayName'] == 'Pass')].copy()

    # Formato SPADL
    df_xT.rename(columns={
        "id":"game_id", "eventId":"event_id", "teamId":"team_id",
        "x": "start_x", "y": "start_y", "endX":"end_x", "endY":"end_y",
        "period_value":"value", "period_displayName":"displayName",
        "type_value": "type_id", "type_displayName": "type_name",
        "outcomeType_value": "result_id", "outcomeType_displayName": "result_name",
        "playerId": "player_id"
    }, inplace=True)

    df_xT['type_name'] = np.where(df_xT['isShot'] == 1, 'shot', 'pass')
    df_xT['type_id'] = np.where(df_xT['isShot'] == 1, 11, 0)
    df_xT['isGoal'] = df_xT.isGoal.fillna(0)
    df_xT['result_name'] = np.where(
        (df_xT['type_name'] == 'shot') & (df_xT['isGoal'] != 1),
        'fail',
        np.where((df_xT['result_id'] != 1) & (df_xT['type_name'] == 'pass'),
                 'Unsuccessful', 'Successful')
    )
    df_xT[['end_x','end_y']] = df_xT[['end_x', 'end_y']].fillna(0)
    df_xT['start_x'] *= 1.05
    df_xT['start_y'] *= 0.68
    df_xT['end_x'] *= 1.05
    df_xT['end_y'] *= 0.68

    # Aplicar modelo
    model = load_model("https://karun.in/blog/data/open_xt_12x8_v1.json")
    df_xT = get_successful_move_actions(df_xT)
    df_xT["xT_value"] = model.rate(df_xT)
    # Eliminar o corregir valores desmesurados
    df_xT.loc[df_xT["xT_value"].abs() > 1, "xT_value"] = 0
    # Unir de vuelta por índice (posición en el dataframe original)
    df['xT_value'] = 0
    df.loc[df_xT.index, 'xT_value'] = df_xT['xT_value'].astype(float)
    return df

#Tiros realizados
def add_shots_and_goals(df):
    df['isShot'] = df['type_displayName'].isin(['Goal', 'Attempt Saved', 'Miss', 'Post']).astype(int)
    df['Shot'] = df['isShot'].fillna(0).astype(int)
    df['Goal'] = df['isGoal'].fillna(0).astype(int)
    return df

#Regates
def add_dribbles(df):
    df['Dribble_Successful'] = (
        (df['type_displayName'] == 'Take On') &
        (df['outcomeType_displayName'] == 'Successful')
    ).astype(int)

    df['Dribble_Unsuccessful'] = (
        (df['type_displayName'] == 'Take On') &
        (df['outcomeType_displayName'] == 'Unsuccessful')
    ).astype(int)
    return df

#Acciones defensivas
def add_defensive_actions(df):
    acciones_defensivas_exitosas = [
    "Aerial",
    "Attempted tackle",
    "Ball recovery",
    "Blocked Pass",
    "Challenge",
    "Clearance",
    "Claim",
    "Dispossessed",
    "Foul",
    "Interception",
    "Offside provoked",
    "Shield ball opp",
    "Tackle"
    ]
    df['DefensiveAction_Successful'] = (
        df['type_displayName'].isin(acciones_defensivas_exitosas) &
        (df['outcomeType_displayName'] == 'Successful')
    ).astype(int)

    df['DefensiveAction_Unsuccessful'] = (
        df['type_displayName'].isin(acciones_defensivas_exitosas) &
        (df['outcomeType_displayName'] == 'Unsuccessful')
    ).astype(int)

    df['Clearance'] = (df['type_displayName'] == 'Clearance').astype(int)

    df['Entries_Successful'] = (
        (df['type_displayName'] == 'Tackle') &
        (df['outcomeType_displayName'] == 'Successful') 
    ).astype(int)

    df['Aerial_Successful'] = (
        (df['type_displayName'] == 'Aerial') &
        (df['outcomeType_displayName'] == 'Successful')
    ).astype(int)

    df['Aerial_Unsuccessful'] = (
        (df['type_displayName'] == 'Aerial') &
        (df['outcomeType_displayName'] == 'Unsuccessful')
    ).astype(int)

    return df

#Parte del cuerpo utilizada por acciones
def add_body_part(df):
    body_parts_cols = {
        'value_HeadPass': 'Head',
        'value_LeftFoot': 'LeftFoot',
        'value_RightFoot': 'RightFoot',
        'value_OtherBodyPart': 'OtherBodyPart',
        'value_Hands': 'Hands'
    }

    def obtener_parte_cuerpo(row):
        for col, label in body_parts_cols.items():
            if pd.notna(row.get(col)) and row.get(col) != 0:
                return label
        return 'Unknown'

    df['body_part'] = df.apply(obtener_parte_cuerpo, axis=1)
    return df

#Duelos
def add_duels(df):
    duels = ['Take On', 'Tackle', 'Aerial', 'Challenge', 'Shield ball opp']
    
    df['Duels_Successful'] = (
        df['type_displayName'].isin(duels) &
        (df['outcomeType_displayName'] == 'Successful')
    ).astype(int)

    df['Duels_Unsuccessful'] = (
        df['type_displayName'].isin(duels) &
        (df['outcomeType_displayName'] == 'Unsuccessful')
    ).astype(int)

    return df

#Accion hacia una transición
def add_defensive_to_transition(df):
    acciones_defensivas_exitosas = [
        'Interception', 'BallRecovery', 'Tackle', 'Blocked Pass',
        'Clearance', 'Aerial', 'Challenge', 'Shield Ball Opp', 'Offside provoked'
    ]

    df['DefensiveToTransition'] = 0

    for i in range(len(df) - 3):
        fila = df.iloc[i]
        if (
            fila['type_displayName'] in acciones_defensivas_exitosas and
            fila['outcomeType_displayName'] == 'Successful'
        ):
            equipo = fila['teamId']
            for j in range(1, 4):  # mirar las 3 siguientes filas
                siguiente = df.iloc[i + j]
                if siguiente['teamId'] == equipo and siguiente['type_displayName'] in ['Pass', 'TakeOn', 'Shot']:
                    df.at[i, 'DefensiveToTransition'] = 1
                    break

    return df

#Eficiencia Ofensiva Tras Presión Alta
def calcular_eficiencia_ofensiva_presion_alta(df):
    df['HighTurnover'] = 0
    df['HighTurnoverShot'] = 0

    acciones_defensivas = ['Interception', 'Ball recovery']

    for i in range(len(df)):
        fila = df.iloc[i]

        if (
            fila['type_displayName'] in acciones_defensivas and
            fila['outcomeType_displayName'] == 'Successful' and
            fila['x'] > 50  # Campo rival
        ):
            equipo = fila['teamId']
            tiempo_inicio = fila['time_seconds']
            df.at[i, 'HighTurnover'] = 1

            # Buscar si en los próximos eventos hay un tiro del mismo equipo en los siguientes 20 segundos
            for j in range(i + 1, min(i + 20, len(df))):  # Limita la búsqueda a los 20 siguientes eventos (ajustable)
                siguiente = df.iloc[j]
                if (
                    siguiente['teamId'] == equipo and
                    siguiente['isShot'] == 1 and
                    siguiente['time_seconds'] - tiempo_inicio <= 20
                ):
                    df.at[i, 'HighTurnoverShot'] = 1
                    break
    return df

#Expected goals
def calcular_xG_modelo(df, ruta_modelo):
    with open(ruta_modelo, 'rb') as f:
        model = pickle.load(f)
    
    b = model.params.values
    model_variables = model.model.exog_names[1:]

    rename_dict = {
        'type_value': 'eventId',
        'playerId': 'playerId',
        'matchId': 'matchId',
        'Shot': 'eventName',
        'teamId': 'teamId',
        'period_value': 'matchPeriod',
        'Segundos': 'eventSec',
        'id': 'id',
        'x': 'X',
        'y': 'Y'
    }
    df['isShot'] = df['type_displayName'].isin(['Goal', 'Attempt Saved', 'Miss', 'Post']).astype(int)
    df['Shot'] = df['isShot'].fillna(0).astype(int)
    df_xG = df.rename(columns=rename_dict).copy()
    df_xG = df_xG[df_xG['eventName'] == 1]

    df_xG["X"] = (100 - df_xG['X']) * 105 / 100
    df_xG["Y"] = df_xG['Y'] * 68 / 100
    df_xG["C"] = abs(df_xG['Y'] - 34)
    df_xG["Distance"] = np.sqrt(df_xG["X"]**2 + df_xG["C"]**2)

    denominator = df_xG["X"]**2 + df_xG["C"]**2 - (7.32 / 2)**2
    angle_raw = np.arctan(7.32 * df_xG["X"] / denominator)
    df_xG["Angle"] = np.where(angle_raw > 0, angle_raw, angle_raw + np.pi)

    df_xG["X2"] = df_xG['X']**2
    df_xG["C2"] = df_xG['C']**2
    df_xG["AX"] = df_xG['Angle'] * df_xG['X']

    def logistic_function(row):
        bsum = b[0]
        for i, var in enumerate(model_variables):
            bsum += b[i + 1] * row[var]
        return 1 / (1 + np.exp(-bsum))
    
    df_xG["xG"] = df_xG.apply(logistic_function, axis=1)

    df = df.merge(df_xG[["id", "xG"]], on="id", how="left")

    return df

In [5]:
ruta_modelo = r"G:\Mi unidad\TFM\OPTA\Modelos\xg_model.pkl"

df_renombrado = preparar_datos(df_renombrado)
df_renombrado = calcular_pases(df_renombrado)
df_renombrado = calcular_toques_area(df_renombrado)
df_renombrado = aplicar_xT(df_renombrado)
df_renombrado = add_shots_and_goals(df_renombrado)
df_renombrado = add_dribbles(df_renombrado)
df_renombrado = add_defensive_actions(df_renombrado)
df_renombrado = add_body_part(df_renombrado)
df_renombrado = add_duels(df_renombrado)
df_renombrado = add_defensive_to_transition(df_renombrado)
df_renombrado = calcular_eficiencia_ofensiva_presion_alta(df_renombrado)
df_renombrado = calcular_xG_modelo(df_renombrado, ruta_modelo)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_25724\1783821751.py:133: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[-0.00352222  0.00082343 -0.00088452 ...  0.00789708 -0.0002083
  0.00206551]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df_xT.index, 'xT_value'] = df_xT['xT_value'].astype(float)


In [6]:
df_renombrado = df_renombrado.drop(columns=['xG_x', 'xG_y', 'Clearence', 'timeMin', 'time_seconds','Xt_value', 'isFirstEleven'], errors='ignore')

In [9]:
columnas_ordenadas = {
    "Identificación de evento y partido": [
        'id', 'matchId', 'eventId', 'type_displayName', 'period_value',
        'minute', 'second', 'Segundos'
    ],

    "Información de jugador y equipo": [
        'playerId', 'playerName', 'teamId', 'teamName',
        'oppositionTeamName', 'position', 'body_part'
    ],

    "Coordenadas (originales y convertidas)": [
        'x', 'y', 'endX', 'endY', 'x_real', 'y_real', 'endx_real', 'endy_real',
        'ZonaX', 'ZonaY', 'Zona_end_X', 'Zona_end_Y'
    ],

    "Etiquetas de jugada (acciones y contexto)": [
        'isGoal', 'isShot', 'Shot', 'Goal',
        'outcomeType_value', 'outcomeType_displayName',
        'value_Angle', 'value_Length', 'value_Zone',
        'value_PassEndX', 'value_PassEndY'
    ],

    "Características del evento (tags y descriptores)": [
        'value_ShotAssist', 'value_KeyPass', 'value_Longball', 'value_LeftFoot',
        'value_RightFoot', 'value_OppositeRelatedEvent', 'value_GoalShotTimestamp',
        'value_Minutes', 'value_Assisted', 'value_Blocked', 'value_CornerTaken',
        'value_Cross', 'value_Defensive', 'value_Foul', 'value_GoalKick',
        'value_Hands', 'value_Head', 'value_HeadPass', 'value_IndividualPlay',
        'value_IntentionalAssist', 'value_KeeperThrow', 'value_LayOff',
        'value_Offensive', 'value_RegularPlay', 'value_RelatedEventId'
    ],

    "Métricas avanzadas calculadas": [
        'LongPass', 'ShortPass', 'KeyPasses', 'ProgressPasses',
        'TouchesRivalArea','xG', 'TouchesOwnArea',
        'xT_value', 
    ],

    "Acciones específicas y transiciones": [
        'Dribble_Successful', 'Dribble_Unsuccessful',
        'DefensiveAction_Successful', 'DefensiveAction_Unsuccessful',
        'Entries_Successful', 'Aerial_Successful', 'Aerial_Unsuccessful',
        'Duels_Successful', 'Duels_Unsuccessful',
        'Clearance', 'DefensiveToTransition',
        'HighTurnover', 'HighTurnoverShot'
    ]
}

# Paso 2: Aplanar la lista y conservar solo columnas existentes en el DataFrame
columnas_final = [col for grupo in columnas_ordenadas.values() for col in grupo if col in df_renombrado.columns]

# Paso 3: Reordenar el DataFrame
df_final = df_renombrado[columnas_final]


In [11]:
# Lista de columnas a modificar
columnas_objetivo = ['x', 'y', 'endX', 'endY', 'x_real', 'y_real', 'endx_real', 'endy_real', 'xG', 'xT_value']

# Asegurarse de que las columnas existen en el DataFrame
columnas_existentes = [col for col in columnas_objetivo if col in df_final.columns]

# Reemplazar el punto por coma solo en las columnas seleccionadas
df_final[columnas_existentes] = df_final[columnas_existentes].applymap(
    lambda x: str(x).replace('.', ',') if pd.notnull(x) else x
)


C:\Users\Usuario\AppData\Local\Temp\ipykernel_25724\3124685672.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_final[columnas_existentes] = df_final[columnas_existentes].applymap(


In [15]:
df_final.to_csv('FAC_EVENT.csv', index=False)